In [ ]:
from IPython.display import clear_output

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Installing Plugins
Installs GPU and CPU libs

In [ ]:
# Add RAPIDS to do training faster
# Run if on GPU

!pip install cudf-cu12 --extra-index-url=https://pypi.nvidia.com
%load_ext cudf.pandas
clear_output()

In [2]:
!pip install rdkit
!pip install optuna
!pip install optuna-integration[xgboost]
!pip install swifter
!pip install tdqm
!pip install shap
!pip install jax[cuda12]
!pip install umap-learn
!pip install pyscf
clear_output()
print('Finished installing pre-requisites')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.2/36.2 MB 44.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16505 sha256=a1580aefcb02c68738a961b24fdf334cae60beba964e747e9ec859a593d6f903
  Stored in directory: /root/.cache/pip/wheels/ef/7f/bd/9bed48f078f3ee1fa75e0b29b6e0335ce1cb03a38d3443b3a3
Successfully built swifter
  Preparing metadata (setup.py) ... done
  Created wheel for tdqm: filename=tdqm-0.0.1-py3-none-any.whl size=1322 sha256=ff1d557356b19d564683230ccbc34bb483c80a92c2699ca3a779721ee706a897
  Stored in directory: /root/.cache/pip/wheels/c8/c7/30/e5935be2cfa6883be72462333edc414d8928f3c78eaabec38a
Successfully built tdqm
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 37.6 MB/s eta 0:00:00:00:0100:01
  Attempt

NameError: name 'clear_output' is not defined

In [3]:
# Calculations Libs
import numpy as np
import cupy as cp
import pandas as pd
import jax
from jax import numpy as jnp
import swifter

# Hardware
import gc

# Disable Warning
import warnings
warnings.filterwarnings('ignore')

# Disable Warning From rdkit
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

# Descriptor Libs
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors3D, Descriptors, Crippen, rdMolDescriptors, MACCSkeys, RDKFingerprint, rdFingerprintGenerator
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit.Avalon import pyAvalonTools
from rdkit.Chem.rdForceFieldHelpers import MMFFOptimizeMoleculeConfs as MMFFOp
from rdkit.Chem.rdForceFieldHelpers import UFFOptimizeMoleculeConfs as UFFOp
from rdkit.Chem.rdForceFieldHelpers import MMFFHasAllMoleculeParams

import pyscf
from pyscf import gto, scf, dft

# Model processing
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

import umap

# Machine Learning Libs
import xgboost
import lightgbm as lgb

# Training Op Libs
import optuna

# Visualize Libs
import matplotlib.pyplot as plt
import seaborn as sns

# Tools Libs
from joblib import Parallel, delayed, dump
from joblib import load as jload
from tqdm.notebook import tqdm

print('Finnished Importing Plugins!')

2025-10-29 06:47:00.759650: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761720421.005380      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761720421.081682      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Finnished Importing Plugins!


# Load Data
Load data from Kaggle into a pandas DataFrame.

In [ ]:
raw_train = pd.read_csv('/kaggle/input/melting-point/train.csv')
raw_test = pd.read_csv('/kaggle/input/melting-point/test.csv')

bradley_df = pd.read_csv(r'/kaggle/input/melting-point-chemical-dataset/BradleyMeltingPointDataset.csv')
bradleyplus_df = pd.read_csv(r'/kaggle/input/melting-point-chemical-dataset/BradleyDoublePlusGoodMeltingPointDataset.csv')

In [ ]:
# Convert from Celcius (mpC) to Kelvin (Tm)
bradley_df['Tm'] = bradley_df['mpC'].apply(lambda x: x + 273)
bradleyplus_df['Tm'] = bradleyplus_df['mpC'].apply(lambda x: x + 273)

# Get only SMILES and Melting Tempature (Tm)
# Merge both bradle datasets
bradley_merge = pd.concat((
    bradley_df[['smiles', 'Tm']],
    bradleyplus_df[['smiles', 'Tm']]
), axis=0)
bradley_merge = bradley_merge.rename(columns = {'smiles': 'SMILES'})

bradley_merge

In [ ]:
# Merge both datasets to one
train_df = pd.concat((
    raw_train[['SMILES', 'Tm']], 
    bradley_merge
), axis=0)

test_df = raw_test[['id', 'SMILES']]

display(train_df, test_df)

## Canonical SMILES
Change all SMILES to they're *canonical* counterparts to remove duplicated samples. The problem with this is difference in samples between datasets, when calculated the *SMILES* counterpart, then the duplicate process runs it detects the same canonical *SMILES* but with different *Tm* (Melting point). We handle this with just choosing one of them. Have not found a better solution for this problem. But, more than 85% of the duplicated data has a variance average of lower than 6° Kelvin. But due to different ways of measureing tempatures in different datasets available, 5% of the samples have large variance more than 25° Kelvin which is dropped due to inconssistent data to not feed the model unrelated data.

The *canonical_smiles_variance* function also calculate the mol and 3d mol for future mol calculations.

In [ ]:
def generate_3d_conformer(mol_2d: Chem.Mol, random_seed: int = 42) -> Chem.Mol | None:
    """
    Robustly generate a 3D conformer for an RDKit Mol object.

    Handles:
      - Hydrogen addition
      - Modern ETKDGv3 embedding
      - **Multiple conformer generation attempts (numConfs=10)**
      - **Optimization of all conformers**
      - MMFF vs UFF optimization fallback
      - **Selection of the lowest-energy conformer**
      - Reproducible random seed

    Returns:
      mol3d (RDKit Mol with one 3D conformer) or None if failed.
    """
    
    # Create a new molecule object and add hydrogens
    # This is crucial for getting a correct 3D geometry
    mol = Chem.AddHs(mol)

    # Set up the modern ETKDGv3 embedding parameters
    params = AllChem.ETKDGv3()
    params.randomSeed = random_seed
    params.useRandomCoords = True # Use random starting coordinates
    params.numThreads = 0 # Use all available CPU cores
    
    # Attempt to embed multiple conformers
    # This is more robust than a single EmbedMolecule call.
    cids = AllChem.EmbedMultipleConfs(mol, numConfs=10, params=params)

    if len(cids) == 0:
        print(f"Failed to embed any conformers for molecule: {Chem.MolToSmiles(mol_2d)}")
        return None

    # Optimize the geometries
    # We prefer the MMFF94 force field, but fall back to UFF.
    # We optimize all generated conformers (cids).
    energies = []
    try:
        if MMFFHasAllMoleculeParams(mol):
            # MMFFOptimizeMoleculeConfs returns list of (not_converged, energy)
            results = MMFFOp(mol, numThreads=0)
            energies = [res for res in results if res[0] == 0] # Filter for converged
        else:
            # Use UFF as a fallback
            results = UFFOp(mol, numThreads=0)
            energies = [res for res in results if res[0] == 0] # Filter for converged

    except Exception as e:
        # Catch any other optimization errors and try UFF
        print(f"MMFF optimization failed ({e}), falling back to UFF.")
        try:
            results = UFFOp(mol, numThreads=0)
            energies = [res for res in results if res[0] == 0] # Filter for converged
        except Exception as e_uff:
            print(f"UFF optimization also failed: {e_uff}")
            # We might still have cids, but optimization failed.
            pass # Continue, we might have some unoptimized energies if needed, or just fail

    if not energies:
        print(f"All conformer optimizations failed for: {Chem.MolToSmiles(mol_2d)}")
        # As a last resort, we could return the unoptimized conformer, 
        # but for robustness, it's better to signal failure.
        return None

    # Find the conformer with the lowest energy
    # energies is a list of (conformer_id, energy)
    sorted_energies = sorted(energies, key=lambda x: x[1])
    best_cid = sorted_energies[0][0]
    best_energy = sorted_energies[0][1]

    # print(f"Found best conformer (ID {best_cid}) with energy: {best_energy:.2f}")

    # Create a new molecule to hold only the best conformer
    final_mol = Chem.Mol(mol)
    final_mol.RemoveAllConformers()
    
    # Get the specific conformer from the original molecule
    best_conf = mol.GetConformer(best_cid)
    final_mol.AddConformer(best_conf, assignId=True)
    
    return final_mol

In [ ]:
def canonical_smiles_variance(canonical_df, target_col='Tm', low_thr=5, high_thr=25):
    df = canonical_df.copy()

    # --- Ensure target column is numeric ---
    df[target_col] = pd.to_numeric(df[target_col], errors='coerce')
    df = df.dropna(subset=[target_col])

    # --- Handle duplicates only ---
    dups = df[df['SMILES'].duplicated(keep=False)]
    uniq = df[~df['SMILES'].duplicated(keep=False)]

    if dups.empty:
        print("No duplicates found after canonicalization.")
        return df, pd.DataFrame()

    stats = (
        dups.groupby('SMILES')[target_col]
        .agg(['mean', 'median', 'std', 'count'])
        .reset_index()
    )

    # --- Merge duplicates based on variance thresholds ---
    stats['value'] = stats.apply(
        lambda r: r['mean'] if r['std'] < low_thr
        else r['median'] if r['std'] <= high_thr
        else None, axis=1
    )

    merged_df = stats.dropna(subset=['value'])[['SMILES', 'value']]
    merged_df = merged_df.rename(columns={'value': target_col})

    # Reattach Mol objects for merged entries
    merged_df['Mol'] = merged_df['SMILES'].map(
        df.drop_duplicates('SMILES').set_index('SMILES')['Mol']
    )

    # Combine merged + unique
    cleaned = pd.concat([uniq[['SMILES', target_col, 'Mol']], merged_df], ignore_index=True)
    cleaned = cleaned.reset_index(drop=True)

    print(f"Original: {df.shape[0]} rows")
    print(f"Unique:   {uniq.shape[0]}")
    print(f"Merged:   {len(merged_df)}")
    print(f"Dropped Varience:  {stats.shape[0] - len(merged_df)} (high variance > {high_thr} K)")

    display(
        cleaned,
        cleaned.info(),
        stats,
        cleaned['SMILES'].isna().value_counts()
    )

    return cleaned, stats

In [ ]:
def canonicalize_smiles_df(df, stat=False):
    """
    Converts SMILES strings in a DataFrame to their canonical form.

    This function performs the following steps:
    1. Creates a copy of the DataFrame.
    2. Converts SMILES strings from 'smiles_col' to RDKit Mol objects
       and stores them in a new 'Mol' column.
    3. Drops any rows where the SMILES string was invalid (Mol is None).
    4. Converts the 'Mol' objects back to canonical SMILES strings,
       updating 'smiles_col' in place.
    
    This is a simplified version of your original function, removing
    all statistics calculation, duplicate merging, and 3D structure generation.

    Args:
        df (pd.DataFrame): The input DataFrame.
        smiles_col (str): The name of the column containing SMILES strings.
                          Defaults to 'SMILES'.

    Returns:
        pd.DataFrame: A new DataFrame with canonical SMILES, a 'Mol' column,
                      and invalid SMILES rows removed.
    """
    # Suppress RDKit errors (like for 'invalid-smiles') during Mol conversion
    
    df_copy = df.copy()

    print("Step 1: Converting SMILES to Mol objects...")
    # --- Convert to Mol objects (keep them) ---
    mol_series = df_copy['SMILES'].swifter.apply(Chem.MolFromSmiles)
    df_copy['Mol'] = mol_series

    # --- Drop rows where Mol conversion failed ---
    initial_rows = df_copy.shape[0]
    df_copy = df_copy.dropna(subset=['Mol']).reset_index(drop=True)
    dropped_rows = initial_rows - df_copy.shape[0]
    
    if dropped_rows > 0:
        print(f"Step 2: Dropped {dropped_rows} rows due to invalid/unparseable SMILES.")
    else:
        print("Step 2: All SMILES strings parsed successfully.")

    print("Step 3: Canonicalizing SMILES strings...")
    # --- Canonicalize SMILES (Mol remains valid) ---
    # Note: isomericSmiles=True is kept from the original to preserve stereochemistry
    canonical_smiles_series = df_copy['Mol'].swifter.apply(
        lambda m: Chem.MolToSmiles(m, canonical=True, isomericSmiles=True)
    )

    df_copy['SMILES'] = canonical_smiles_series
    
    print("Canonicalization complete.")

    #print("Step 4: Generating 3D Mol...")
    #df_copy['Mol3D'] = df_copy['Mol'].apply(generate_3d_conformer)
    #df_copy = df_copy[df_copy['Mol3D'].notnull()]
    #df_copy = df_copy.reset_index(drop=True)
    if stat == True:
        df_copy, stats = canonical_smiles_variance(df_copy)
        return df_copy, stats
    else:
        return df_copy

In [ ]:
train_df, stats = canonicalize_smiles_df(train_df, stat=True)
test_df = canonicalize_smiles_df(test_df)

### Visualize variance of melting tempature of duplicated SMILES

In [ ]:
def plot_tm_variance_colored(variance_df):
    stds = variance_df['std'].dropna()

    # Define bins
    bins = np.linspace(0, stds.max(), 60)

    plt.figure(figsize=(10, 6))
    n, bins, patches = plt.hist(stds, bins=bins, edgecolor='black', alpha=0.8)

    # Color-code regions
    for patch, left_edge in zip(patches, bins[:-1]):
        if left_edge < 5:
            patch.set_facecolor('green')
        elif left_edge < 25:
            patch.set_facecolor('gold')
        else:
            patch.set_facecolor('red')

    plt.title('Variance in Melting Points Across Duplicate Canonical SMILES')
    plt.xlabel('Standard Deviation (K)')
    plt.ylabel('Number of Molecules')
    plt.axvline(5, color='black', linestyle='--', label='5 K (good threshold)')
    plt.axvline(25, color='red', linestyle='--', label='25 K (large variance)')
    plt.legend()
    plt.grid(alpha=0.3)
    plt.show()

def summarize_variance_plot(variance_df, show_pie=False):
    # Clean data
    stds = variance_df['std'].dropna()
    total = len(stds)

    # Define categories
    categories = {
        'Excellent (<5 K)': (stds < 5).sum(),
        'Moderate (5–25 K)': ((stds >= 5) & (stds <= 25)).sum(),
        'High (>25 K)': (stds > 25).sum()
    }

    # Convert to percentages
    labels = list(categories.keys())
    values = [v / total * 100 for v in categories.values()]
    
    # --- Visualization ---
    plt.figure(figsize=(7, 5))

    if show_pie:
        # Pie chart
        plt.pie(
            values,
            labels=labels,
            autopct='%1.1f%%',
            startangle=140,
            colors=['green', 'gold', 'red']
        )
        plt.title('Melting Point Variance Summary (by Category)')
    else:
        # Bar chart
        plt.bar(labels, values, color=['green', 'gold', 'red'], edgecolor='black')
        plt.title('Melting Point Variance Summary')
        plt.ylabel('Percentage of Duplicates (%)')
        plt.grid(axis='y', alpha=0.3)

    plt.tight_layout()
    plt.show()

summarize_variance_plot(stats, show_pie=True)
plot_tm_variance_colored(stats)

## Calculate the Descriptors of the molecules.

In [ ]:
def extract_2d_desc(df: pd.DataFrame) -> pd.DataFrame:
    if 'Mol' not in df.columns:
        df['Mol'] = df['SMILES'].swifter.apply(Chem.MolFromSmiles)
    desc_features = pd.DataFrame(
        Parallel(n_jobs=-1)(
            delayed(Descriptors.CalcMolDescriptors)(mol) for mol in tqdm(df['Mol'], desc="Calculating 2D descriptors")
        )
    )
    
    return pd.concat([df, desc_features], axis=1)

In [ ]:
def extract_3d_desc(df: pd.DataFrame) -> pd.DataFrame:
    if 'Mol3D' not in df.columns:
        df['Mol3D'] = df['Mol'].swifter.apply(generate_3d_mol)
        df = df[df['Mol3D'].notnull()]
    desc_features = pd.DataFrame(
        Parallel(n_jobs=-1)(
            delayed(Descriptors3D.CalcMolDescriptors3D)(mol) for mol in tqdm(df['Mol3D'], desc="Calculating 3D descriptors")
        )
    )
    
    return pd.concat([df, desc_features], axis=1)

In [ ]:
def extract_fingerprint(df: pd.DataFrame, morgan_radius = 2, morgan_nbits = 1024, train=False) -> pd.DataFrame:
    if 'Mol' not in df.columns:
        df['Mol'] = df['SMILES'].apply(Chem.MolFromSmiles)

    # DEFINE MORGAN GENERATOR
    morgan_gen = rdFingerprintGenerator.GetMorganGenerator(
        radius = morgan_radius, 
        fpSize = morgan_nbits, 
        countSimulation = True, 
        includeChirality = False
    )

    fcfp_gen = rdFingerprintGenerator.GetMorganGenerator(
        radius = morgan_nbits, 
        fpSize = morgan_nbits, 
        atomInvariantsGenerator = rdFingerprintGenerator.GetMorganFeatureAtomInvGen(), 
        countSimulation= True, 
        includeChirality = False
    )

    atom_gen = rdFingerprintGenerator.GetAtomPairGenerator(
        fpSize = 2048, 
        countSimulation= True, 
        includeChirality = False
    )

    # Helper: convert a single molecule into all fingerprints
    def get_all_fps(mol):
        if mol is None:
            # Return zero arrays to preserve shape
            return [
                np.zeros(morgan_nbits),
                np.zeros(morgan_nbits),
                np.zeros(167),        # MACCS
                np.zeros(2048),
                np.zeros(2048),
                np.zeros(morgan_nbits)
            ]
        return [
            morgan_gen.GetFingerprintAsNumPy(mol),
            fcfp_gen.GetFingerprintAsNumPy(mol),
            np.array(list(MACCSkeys.GenMACCSKeys(mol).ToBitString()), dtype=np.uint8),
            atom_gen.GetCountFingerprintAsNumPy(mol),
            np.fromiter(RDKFingerprint(mol).ToBitString(), dtype=np.uint8),
            pyAvalonTools.GetAvalonFP(mol, morgan_nbits)
        ]
    
    # 1. Convert each fingerprint Series to a NumPy array
    fps = df['Mol'].swifter.apply(get_all_fps)

    # 2. Combine all NumPy arrays horizontally in one operation 🚀
    # np.hstack is extremely fast and memory-efficient
        # Stack each fingerprint type separately
    morgan_arr = np.vstack([f[0] for f in fps])
    fcfp_arr = np.vstack([f[1] for f in fps])
    maccs_arr = np.vstack([f[2] for f in fps])
    atom_arr = np.vstack([f[3] for f in fps])
    rdkit_arr = np.vstack([f[4] for f in fps])
    avalon_arr = np.vstack([f[5] for f in fps])

    # Combine all arrays horizontally
    combined_arr = np.hstack([morgan_arr, fcfp_arr, maccs_arr, atom_arr, rdkit_arr, avalon_arr])
    
    # Column Names
    combined_cols = [
        [f'morg_{i}' for i in range(morgan_arr.shape[1])] +
        [f'fcfp_{i}' for i in range(fcfp_arr.shape[1])] +
        [f'maccs_{i}' for i in range(maccs_arr.shape[1])] +
        [f'atom_{i}' for i in range(atom_arr.shape[1])] +
        [f'rdkit_{i}' for i in range(rdkit_arr.shape[1])] +
        [f'avalon_{i}' for i in range(avalon_arr.shape[1])]
    ]

    if train:
        umap_transformer = umap.UMAP(
            n_components=100,
            n_neighbors=15,
            min_dist=0.1,
            random_state=42, # Always set a seed for reproducibility!
            metric='jaccard', # Binary Metric https://umap-learn.readthedocs.io/en/latest/parameters.html#metric
        )
        
        umap_transformer.fit(combined_arr)
        dump(umap_transformer, 'fp_umap.pkl', compress=3)
    else:
        umap_transformer = jload('fp_umap.pkl')
    
    ca_UMAP = umap_transformer.transform(combined_arr)
    ca_Col = [f'umap_{i}' for i in range(100)]
    
    print(f"Original shape: {combined_arr.shape}")
    print(f"New UMAP feature shape: {ca_UMAP.shape}")
    
    # 4. Create the final DataFrame from the combined array and column names
    return pd.concat([df, pd.DataFrame(ca_UMAP, columns=ca_Col)], axis=1)

## *Analyzing fastest apply method :3*
The test was done with the *get_2D_desc*
### Normal .apply()
    CPU times: user 3min 50s, sys: 2.14 s, total: 3min 52s
    Wall time: 2min 38s

### Swifter .apply()
    CPU times: user 4min 5s, sys: 2.03 s, total: 4min 7s
    Wall time: 2min 44s

### List comprehension
    CPU times: user 3min 45s, sys: 787 ms, total: 3min 46s
    Wall time: 2min 32s

### List comprehension Multithreaded:
    CPU times: user 11.4 s, sys: 1.14 s, total: 12.6 s
    Wall time: 59.9 s

In [ ]:
def removena(df):
    # DROP NULL/NONE VALUE AFTER FEATURE ENGINEERING
    return df.copy().dropna().reset_index(drop = True)

In [ ]:
%%time
final_train_df = train_df.copy()
final_test_df = test_df.copy()

final_train_df = extract_2d_desc(final_train_df)
final_test_df = extract_2d_desc(final_test_df)

# final_train_df = extract_3d_desc(final_train_df)
# final_test_df = extract_3d_desc(final_test_df)

# Remember to train first
final_train_df = extract_fingerprint(final_train_df, train=True)
final_test_df = extract_fingerprint(final_test_df)

final_train_df = removena(final_train_df)
final_test_df = removena(final_test_df)

In [ ]:
display(
    final_train_df,
    final_train_df.info(),
    final_test_df, 
    final_test_df.info()
)

In [ ]:
# DROP DUPLICATED FROM MERGED DATA
def drop_dup(df):
    display(f'There are {df.duplicated(subset = ["SMILES", "Tm"]).sum()} Duplicated data')
    df = df.drop_duplicates(subset = ['SMILES', 'Tm']).reset_index(drop = True)
    print(f'Successfully dropped Duplicated data based on SMILES and Tm!\n\n')
    
    display(f'There are {df.duplicated(subset = ["SMILES"]).sum()} duplicated rows based on SMILES')
    df = df.drop_duplicates(subset = ['SMILES'], keep='first').reset_index(drop = True)
    print(f'Successfully dropped duplicated rows based on SMILES!')

    return df

# Dont really need to use!

In [ ]:
def reduce_memory_usage(df: pd.DataFrame, use_unsigned=True, verbose=True) -> pd.DataFrame:
    """
    Downcast numeric columns in a DataFrame to the smallest possible dtype.
    Optionally converts non-negative integers to unsigned types.
    
    Parameters
    ----------
    df : pd.DataFrame
        Input DataFrame
    use_unsigned : bool
        Whether to allow unsigned integer types (uint8/16/32/64) when all values >= 0
    verbose : bool
        Whether to print memory usage statistics
    
    Returns
    -------
    pd.DataFrame
        Memory-optimized DataFrame
    """
    start_mem = df.memory_usage(deep=True).sum() / 1024**2
    if verbose:
        print(f"Initial memory usage: {start_mem:.2f} MB")

    for col in df.select_dtypes(include=[np.number]).columns:
        try:
            col_type = df[col].dtype
        except:
            display('dtype error skipping...')
            return df
        c_min, c_max = df[col].min(), df[col].max()

        if np.issubdtype(col_type, np.integer):
            # Use unsigned types when possible
            if use_unsigned and c_min >= 0:
                if c_max < np.iinfo(np.uint8).max:
                    df[col] = df[col].astype(np.uint8)
                elif c_max < np.iinfo(np.uint16).max:
                    df[col] = df[col].astype(np.uint16)
                elif c_max < np.iinfo(np.uint32).max:
                    df[col] = df[col].astype(np.uint32)
                else:
                    df[col] = df[col].astype(np.uint64)
            else:
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                else:
                    df[col] = df[col].astype(np.int64)

        elif np.issubdtype(col_type, np.floating):
            df[col] = pd.to_numeric(df[col], downcast="float")

    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    if verbose:
        reduction = 100 * (start_mem - end_mem) / start_mem
        print(f"Reduced memory usage: {end_mem:.2f} MB ({reduction:.1f}% reduction)")

    return df

In [ ]:
X = final_train_df.drop(columns=['SMILES', 'Tm', 'Mol'])
y = final_train_df['Tm']

sub = final_test_df.drop(columns=['SMILES', 'id', 'Mol'])
sub = sub.reindex(columns=X.columns, fill_value=0)
assert all(sub.columns == X.columns), "Column order mismatch!"
print("✅ Columns are now aligned with training data.")

In [ ]:
X.columns = X.columns.astype(str)

selector = VarianceThreshold(threshold=0.01).fit(X)


features_mask = selector.get_support()
features_names = X.columns[features_mask]

X_sel_np = selector.transform(X)
sub_sel_np = selector.transform(sub)

print('Finishing variance filter...')

X = pd.DataFrame(X_sel_np, columns=features_names)
sub = pd.DataFrame(sub_sel_np, columns=features_names)

X = reduce_memory_usage(X)
sub = reduce_memory_usage(sub)

X.to_hdf('training.h5', key='data', mode='w', format='table', index=False)
y.to_hdf('test.h5', key='data', mode='w', format='table', index=False)

sub.to_hdf('submission.h5', key='data', mode='w', format='table', index=False)

print('Finishing saving...')

# Jax eval function
Use JAX to use the *just-in-time* (**JIT**) feature to evaluate the metrics faster. I don't what are the overhead of using JAX in this case.

In [ ]:
class metrics:
    class regression:
        """Regression metrics (MAE, MSE, RMSE, R²)."""

        @staticmethod
        @jax.jit
        def mean_absolute_error(
            y_pred: jnp.ndarray, y_true: jnp.ndarray
        ) -> jnp.ndarray:
            """Mean Absolute Error (MAE)"""
            return jnp.mean(jnp.abs(y_pred - y_true))

        @staticmethod
        @jax.jit
        def mean_squared_error(y_pred: jnp.ndarray, y_true: jnp.ndarray) -> jnp.ndarray:
            """Mean Squared Error (MSE)"""
            return jnp.mean((y_pred - y_true) ** 2)

        @staticmethod
        @jax.jit
        def root_mean_squared_error(
            y_pred: jnp.ndarray, y_true: jnp.ndarray
        ) -> jnp.ndarray:
            """Root Mean Squared Error (RMSE)"""
            return jnp.sqrt(jnp.mean((y_pred - y_true) ** 2))

        @staticmethod
        @jax.jit
        def r2_score(y_pred: jnp.ndarray, y_true: jnp.ndarray) -> jnp.ndarray:
            """Coefficient of Determination (R²)"""
            ss_res = jnp.sum((y_true - y_pred) ** 2)
            ss_tot = jnp.sum((y_true - jnp.mean(y_true)) ** 2)
            return jnp.where(ss_tot == 0, 0.0, 1 - ss_res / ss_tot)

In [ ]:
from sklearn.metrics import mean_squared_error
import timeit

x1 = np.random.rand(1, 200)
x2 = np.random.rand(1, 200)

In [ ]:
%%time
t = timeit.timeit(lambda: metrics.regression.root_mean_squared_error(jnp.array(x1), jnp.array(x2)), number=1000)
print("Avg time per run:", t / 1000)

In [ ]:
%%time
t = timeit.timeit(lambda: np.sqrt(mean_squared_error(x1, x2)), number=1000)
print("Avg time per run:", t / 1000)

# Use a gpu from this point onwards!!!
We will use the cuda cores so if you are still in a *CPU session* this will not work!!!

In [ ]:
xgboost.set_config(enable_pinned_memory=True)

In [ ]:
X = pd.read_hdf('training.h5', key='data')
y = pd.read_hdf('test.h5', key='data')

display(X, y)

In [ ]:
from optuna.integration import XGBoostPruningCallback

def objective(trial) -> float:
    param = {
        'verbosity': 0,
        'objective': 'reg:squarederror',
        'device': 'cuda', # USE CUDA ACCELERATION
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor',
        'n_gpus': -1,
        'eval_metric': 'rmse',
        'n_estimators': 10_000,
        'booster': 'gbtree',
        "grow_policy": "lossguide",  # often improves chem-space generalization
        
        'learning_rate': trial.suggest_float('learning_rate', 3e-3, 0.3),
        'max_depth': trial.suggest_int('max_depth', 4,9),
        
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 6),
        'max_delta_step': trial.suggest_float('max_delta_step', 0, 10),
        
        'subsample': trial.suggest_float('subsample', 0.65, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.65, 1.0),
        
        'gamma': trial.suggest_float('gamma', 0, 3),
        'lambda': trial.suggest_float('lambda', 0.1, 20.0, log=True),
        'alpha': trial.suggest_float('alpha', 0.1, 20.0, log=True),
    }

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    kf_indices = list(kf.split(X))

    rmse_scores = []

    with xgboost.config_context(use_rmm=True):
        for train_indices, val_indices in kf_indices:

            X_train_fold, X_val_fold = X.iloc[train_indices], X.iloc[val_indices]
            y_train_fold, y_val_fold = y[train_indices], y[val_indices]
    
            dtrain = xgboost.DMatrix(X_train_fold, label=y_train_fold)
            dvalid = xgboost.DMatrix(X_val_fold, label=y_val_fold)
    
            xgb = xgboost.train(
                params=param,
                dtrain=dtrain,
                num_boost_round=10000,
                evals=[(dtrain, 'training'), (dvalid, 'validation')],
                early_stopping_rounds=int(np.clip(100 / param["learning_rate"], 20, 100)),
                verbose_eval=False,
            )
    
            rmse_scores.append(xgb.best_score)

            del xgb # Free gpu memory
            gc.collect()
            cp.get_default_memory_pool().free_all_blocks()
    
    # Return mean float
    return float(np.mean(rmse_scores))

In [ ]:
# CLEAR OUTPUT AFTER TRAINING 
clear_output(wait=True)

sampler = optuna.samplers.TPESampler(
    multivariate=True,      # Uses correlations between params
    seed=42                 # Reproducible
)

pruner = optuna.pruners.MedianPruner(
    n_startup_trials=3,     # Let first few trials finish
    n_warmup_steps=50,      # Wait before pruning
    interval_steps=20       # Check pruning every N steps
)

study = optuna.create_study(
    study_name='xgboost_study',
    direction='minimize',
    sampler=sampler,
    pruner=pruner,
    storage="sqlite:///xgb_study.db",   # cleaner path
    load_if_exists=True
)

study.optimize(
    objective, 
    n_trials=50,                 # increase trials for real tuning
    timeout=60 * 60 * 4,         # x-hour cap
    show_progress_bar=True,
    gc_after_trial=True,         # free GPU memory after each trial
    n_jobs=1                    # n_jobs set to available GPU's
)

print('Training Completed! Here are the insights:')
best_params = study.best_params

print(f'Total trials: {len(study.trials)}\n')

print("Best Trial", study.best_trial.number)
print("Best RMSE:", study.best_value)
print("Best Params:", study.best_trial.params)

In [ ]:
# Specify the path to the output file
file_path = 'xgb_study.db.'

# Check if the file exists
if os.path.exists(file_path):
    # Remove the file
    os.remove(file_path)
    print(f"{file_path} has been deleted.")
else:
    print(f"{file_path} does not exist.")

In [5]:
from plotly.io import show

loaded_study = optuna.load_study(study_name="xgboost_study", storage="sqlite:///xgb_study.db")
flg = optuna.visualization.plot_timeline(loaded_study)
show(flg)

In [ ]:
fig = optuna.visualization.plot_param_importances(loaded_study)
show(fig)
importance = pd.DataFrame({
    'Name': fig.data[0]['y'],
    'Importance': fig.data[0]['x']
}).sort_values(by=['Importance'], ascending=False).reset_index(drop=True)

In [ ]:
flg = optuna.visualization.plot_optimization_history(loaded_study)
show(flg)

In [9]:
loaded_study.best_params

{'learning_rate': 0.16186921257363845,
 'max_depth': 6,
 'min_child_weight': 6,
 'max_delta_step': 0.26624448580987936,
 'subsample': 0.8897218397722685,
 'colsample_bytree': 0.7188172466872444,
 'gamma': 2.4632570628465293,
 'lambda': 2.338564224704382,
 'alpha': 1.2430282936492325}

In [ ]:
flg = optuna.visualization.plot_contour(
    loaded_study, 
    params=['max_delta_step', 'max_depth'], 
    target_name='RMSE'
)
show(flg)

# UMAP Paramater Visualization

In [ ]:
rows = []
for trial in loaded_study.trials:
    if trial.state == optuna.trial.TrialState.COMPLETE:
        row = {
            'trial_number': trial.number,
            'value': trial.value,
            'state': trial.state.name,
            'duration': trial.duration.total_seconds() if trial.duration else None
        }
        # Merge parameters
        row.update(trial.params)
        rows.append(row)

df = pd.DataFrame(rows)
df.sort_values(['colsample_bytree'], ascending=True).reset_index().head(10)

In [ ]:
## import pandas as pd
from umap import UMAP
from sklearn.preprocessing import StandardScaler
import plotly.express as px

# --- Prepare features ---
X = df.drop(columns=['trial_number', 'state', 'duration', 'value'], errors='ignore')
y = df['value']

# Optional: log-transform positive-only parameters
for c in X.columns:
    if (X[c] > 0).all() and X[c].max() / X[c].min() > 100:
        X[c] = np.log1p(X[c])

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# --- Tune UMAP for your dataset ---
umap = UMAP(
    n_components=2,
    n_neighbors=10,     # smaller -> focus on local clusters, larger -> global trends
    min_dist=0.1,       # larger -> more spread out, smaller -> tighter clusters
    metric='euclidean',
    random_state=42
)
X_embedded = umap.fit_transform(X_scaled)

# --- Plot ---
vis_df = pd.DataFrame({
    'umap_x': X_embedded[:, 0],
    'umap_y': X_embedded[:, 1],
    'RMSE': y
})
import numpy as np
import plotly.express as px

# We'll assume you already have vis_df with umap_x, umap_y, and RMSE columns
best_threshold = np.percentile(vis_df['RMSE'], 5)  # top 10% best (lowest RMSE)

vis_df['is_best'] = vis_df['RMSE'] <= best_threshold

fig = px.scatter(
    vis_df,
    x='umap_x',
    y='umap_y',
    color='RMSE',
    color_continuous_scale='Viridis_r',  # reverse so low RMSE = bright
    symbol='is_best',
    symbol_map={True: 'star', False: 'circle'},
    size=np.where(vis_df['is_best'], 10, 2),
    opacity=0.8,
    title='UMAP Visualization of Optuna Trials (Best Results Highlighted)',
    hover_data=['RMSE']
)
fig.show()